In [1]:
# Importamos las librerias que vamos a usar para hacer la limpieza de los datos y también drive de google.colab para poder hacer la lectura de los datos que están en nuestra nube.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Establecemos el dataframe y solicitamos que nos de una pequeña muestra de los datos
df = pd.read_csv("/content/drive/MyDrive/biciMAD/bicimad_stations_data.csv")
df.head()

,timestamp,station_id,name,address,longitude,latitude,total_bases,active_bases,available_bikes,free_bases,reservations,status
0,2025-04-08 09:00:01,1409,5 - Fuencarral,"Calle Fuencarral n. 106,",-3.702135,40.428521,27,27,17,10,0,1
1,2025-04-08 09:00:01,1412,8 - Metro Alonso Martinez,"Plaza de Alonso Martinez n. 6,",-3.695440,40.427868,24,24,7,17,0,1
2,2025-04-08 09:00:01,1414,10 - Marques de la Ensenada,"Calle Marques de la Ensenada (junto a edif, Ce...",-3.692104,40.425403,23,23,3,20,0,1
3,2025-04-08 09:00:01,1415,11 - Plaza del Dos de Mayo,"Calle San Andres n. 18 - 20,",-3.703600,40.427000,23,23,1,22,0,1
4,2025-04-08 09:00:01,2286,609 - Metro Ronda de la Comunicacion,"Ronda de la Comunicacion, s/n,",-3.662689,40.515720,23,23,1,22,0,1


In [3]:
#Verificamos los tipos de datos del df
df.dtypes

,0
timestamp,object
station_id,int64
name,object
address,object
longitude,float64
latitude,float64
total_bases,int64
active_bases,int64
available_bikes,int64
free_bases,int64


In [4]:
#Tambien vemos la descripción de los datos
df.describe()

,station_id,longitude,latitude,total_bases,active_bases,available_bikes,free_bases,reservations,status
count,53210.000000,53210.000000,53210.000000,53210.000000,53210.000000,53210.000000,53210.000000,53210.0,53210.0
mean,1908.801128,-3.682427,40.424730,23.819846,23.626818,10.873896,12.752922,0.0,1.0
std,323.324990,0.042515,0.036026,2.340746,4.193883,7.886593,7.073843,0.0,0.0
min,1406.000000,-3.784627,40.332546,12.000000,0.000000,0.000000,0.000000,0.0,1.0
25%,1571.000000,-3.708657,40.398797,23.000000,23.000000,4.000000,7.000000,0.0,1.0
50%,2012.500000,-3.689425,40.423900,23.000000,23.000000,10.000000,13.000000,0.0,1.0
75%,2173.000000,-3.660003,40.449100,24.000000,26.000000,16.000000,19.000000,0.0,1.0
max,2397.000000,-3.548534,40.515720,43.000000,54.000000,50.000000,38.000000,0.0,1.0


In [5]:
# Convertimos los formato de datos en los correspondientes
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
print(df.dtypes)

timestamp          datetime64[ns]
station_id                  int64
name                       object
address                    object
longitude                 float64
latitude                  float64
total_bases                 int64
active_bases                int64
available_bikes             int64
free_bases                  int64
reservations                int64
status                      int64
dtype: object


In [6]:
# Vemos que en la columna "name" hay un número antes del guión que anuncia el nombre de la estación, de acuerdo con BiciMAD, el número antes del guión es el Número publico de la estación, por lo que crearemos una nueva columna para separar ese número.

public_station_id = []
nombres_limpios = []

for nombre in df['name']:
    if ' - ' in nombre:
        partes = nombre.split(' - ', 1)
        nombres_limpios.append(partes[1])
        public_station_id.append(partes[0])
    elif nombre and nombre[0].isdigit():
        encontrado_espacio = False
        for i, char in enumerate(nombre):
            if char == ' ':
                nombres_limpios.append(nombre[i+1:])
                public_station_id.append(nombre[:i])
                encontrado_espacio = True
                break
        if not encontrado_espacio:
            nombres_limpios.append(nombre)
            public_station_id.append(None)
    else:
        nombres_limpios.append(nombre)
        public_station_id.append(None)

df['public_station_id'] = public_station_id
df['name'] = nombres_limpios

print(df[['name', 'public_station_id']])
print(df.dtypes) # Para verificar el tipo de dato de la nueva columna

                                    name public_station_id
0                             Fuencarral                 5
1                  Metro Alonso Martinez                 8
2                 Marques de la Ensenada                10
3                  Plaza del Dos de Mayo                11
4         Metro Ronda de la Comunicacion               609
...                                  ...               ...
53205        Diego Ayllon - Arturo Soria               481
53206        Metro Ligero - Palas de Rey               607
53207        Metro Parque de Santa Maria               531
53208  Mercado Municipal de Vallehermoso               127
53209       Puerto Serrano - Embajadores               265

[53210 rows x 2 columns]
timestamp            datetime64[ns]
station_id                    int64
name                         object
address                      object
longitude                   float64
latitude                    float64
total_bases                   int64
active_bases  

In [7]:
#Ahora procederemos a mover la nueva columna que hemos creado al lado de la columna name
new_column = df.pop('public_station_id')
df.insert(2, 'public_station_id', new_column)
df.head()

,timestamp,station_id,public_station_id,name,address,longitude,latitude,total_bases,active_bases,available_bikes,free_bases,reservations,status
0,2025-04-08 09:00:01,1409,5,Fuencarral,"Calle Fuencarral n. 106,",-3.702135,40.428521,27,27,17,10,0,1
1,2025-04-08 09:00:01,1412,8,Metro Alonso Martinez,"Plaza de Alonso Martinez n. 6,",-3.695440,40.427868,24,24,7,17,0,1
2,2025-04-08 09:00:01,1414,10,Marques de la Ensenada,"Calle Marques de la Ensenada (junto a edif, Ce...",-3.692104,40.425403,23,23,3,20,0,1
3,2025-04-08 09:00:01,1415,11,Plaza del Dos de Mayo,"Calle San Andres n. 18 - 20,",-3.703600,40.427000,23,23,1,22,0,1
4,2025-04-08 09:00:01,2286,609,Metro Ronda de la Comunicacion,"Ronda de la Comunicacion, s/n,",-3.662689,40.515720,23,23,1,22,0,1


In [8]:
#Eliminamos espacios y comas de la columna address

df['address'] = df['address'].str.strip().str.rstrip(',')
df.head()

,timestamp,station_id,public_station_id,name,address,longitude,latitude,total_bases,active_bases,available_bikes,free_bases,reservations,status
0,2025-04-08 09:00:01,1409,5,Fuencarral,Calle Fuencarral n. 106,-3.702135,40.428521,27,27,17,10,0,1
1,2025-04-08 09:00:01,1412,8,Metro Alonso Martinez,Plaza de Alonso Martinez n. 6,-3.695440,40.427868,24,24,7,17,0,1
2,2025-04-08 09:00:01,1414,10,Marques de la Ensenada,"Calle Marques de la Ensenada (junto a edif, Ce...",-3.692104,40.425403,23,23,3,20,0,1
3,2025-04-08 09:00:01,1415,11,Plaza del Dos de Mayo,Calle San Andres n. 18 - 20,-3.703600,40.427000,23,23,1,22,0,1
4,2025-04-08 09:00:01,2286,609,Metro Ronda de la Comunicacion,"Ronda de la Comunicacion, s/n",-3.662689,40.515720,23,23,1,22,0,1


In [9]:
#De acuerdo al log de la recolección de los datos, al principio de la recolección captaba la información de 628 estaciones de anclaje, luego hubieron ciertas alteraciones en la cantidad de estaciones recolectadas a 627, 624 y finalmente a 625 estaciones.
#Vamos a verificar cuales son las estaciones que desparecieron del archivo CSV
todas_estaciones = df['public_station_id'].unique()
print(f"El total de estaciones es: {len(todas_estaciones)}")

El total de estaciones es: 628


In [10]:
#Ahora separaremos los tiempos de recolección y los ordenaremos y sabremos cuantas recolecciones se hicieron durante el periodo de recolección
timestamps = df['timestamp'].unique()
print(f"El total de recolecciones es: {len(timestamps)}")
sorted(timestamps)

El total de recolecciones es: 85


[Timestamp('2025-04-08 09:00:01'),
 Timestamp('2025-04-08 11:00:01'),
 Timestamp('2025-04-08 13:00:03'),
 Timestamp('2025-04-08 15:00:04'),
 Timestamp('2025-04-08 17:00:05'),
 Timestamp('2025-04-08 19:00:07'),
 Timestamp('2025-04-08 21:00:08'),
 Timestamp('2025-04-08 23:00:09'),
 Timestamp('2025-04-09 01:00:10'),
 Timestamp('2025-04-09 03:00:12'),
 Timestamp('2025-04-09 05:00:13'),
 Timestamp('2025-04-09 07:00:15'),
 Timestamp('2025-04-09 09:00:18'),
 Timestamp('2025-04-09 11:00:20'),
 Timestamp('2025-04-09 13:00:21'),
 Timestamp('2025-04-09 15:00:24'),
 Timestamp('2025-04-09 17:00:26'),
 Timestamp('2025-04-09 19:00:31'),
 Timestamp('2025-04-09 21:00:34'),
 Timestamp('2025-04-09 23:00:35'),
 Timestamp('2025-04-10 01:00:36'),
 Timestamp('2025-04-10 03:00:37'),
 Timestamp('2025-04-10 05:00:39'),
 Timestamp('2025-04-10 07:00:40'),
 Timestamp('2025-04-10 09:00:41'),
 Timestamp('2025-04-10 11:00:42'),
 Timestamp('2025-04-10 13:00:43'),
 Timestamp('2025-04-10 15:00:45'),
 Timestamp('2025-04-

In [11]:
# Vamos a analizar las ultimas 10 recolecciones
ultimas_recolecciones = timestamps[-10:]
todas_estaciones = set(df['public_station_id'].unique())

print("Análisis de las últimas 10 recolecciones:")
for ts in ultimas_recolecciones:
    estaciones_recogidas = set(df[df['timestamp'] == ts]['public_station_id'].unique())
    estaciones_perdidas = todas_estaciones - estaciones_recogidas

    print(f"{ts}: {len(estaciones_recogidas)} estaciones recogidas, {len(estaciones_perdidas)} estaciones perdidas")
    if estaciones_perdidas:
        print(f"  Estaciones perdidas: {estaciones_perdidas}")

Análisis de las últimas 10 recolecciones:
2025-04-14 15:01:43: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-14 17:01:44: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-14 19:01:45: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-14 21:01:47: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-14 23:01:48: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-15 01:01:49: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-15 03:01:51: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-15 05:01:52: 624 estaciones recogidas, 4 estaciones perdidas
  Estaciones perdidas: {'31', '36', '32', '33'}
2025-04-15 07:01:53: 6

In [12]:
#Ahora proceremos a analizar cuantas veces se ausentan estas estaciones en la recolección
todas_las_recolecciones = timestamps[-85:]
contador_estaciones_perdidas = {}

for estacion_id in todas_estaciones:
    contador_estaciones_perdidas[estacion_id] = 0

for ts in todas_las_recolecciones:
    estaciones_recogidas = set(df[df['timestamp'] == ts]['public_station_id'].unique())
    estaciones_perdidas = todas_estaciones - estaciones_recogidas

    for estacion_id in estaciones_perdidas:
        contador_estaciones_perdidas[estacion_id] += 1

#Ordernamos las estaciones faltantes por frecuencia de ausencia:

estaciones_ausentes = sorted(contador_estaciones_perdidas.items(), key=lambda x: x[1], reverse=True)

#Presentamos el análisis:
print("Estaciones con más ausencias durante el periodo de recolección:")
for estacion_id, ausencias in estaciones_ausentes:
      if ausencias > 0:  # Solo mostrar las que tienen ausencias
        # Obtener información de la estación
        info_estacion = df[df['public_station_id'] == estacion_id].iloc[0]
        print(f"Estación ID {estacion_id} - {info_estacion['name']}")
        print(f"  Dirección: {info_estacion['address']}")
        print(f"  Ausente en {ausencias} de {len(timestamps)} recolecciones")
        print(f"  Última vez vista: {df[df['public_station_id'] == estacion_id]['timestamp'].max()}")

Estaciones con más ausencias durante el periodo de recolección:
Estación ID 33 - Puerta del Sol
  Dirección: Calle Carretas n. 3
  Ausente en 46 de 85 recolecciones
  Última vez vista: 2025-04-11 13:00:58
Estación ID 31 - Mayor
  Dirección: Calle Mayor n. 16
  Ausente en 46 de 85 recolecciones
  Última vez vista: 2025-04-11 13:00:58
Estación ID 32 - Plaza de la Provincia
  Dirección: Plaza de la Provincia (esquina calle Gerona)
  Ausente en 46 de 85 recolecciones
  Última vez vista: 2025-04-11 13:00:58
Estación ID 265 - Puerto Serrano - Embajadores
  Dirección: Calle Puerto Serrano, 1 enfrente del n. 6 
  Ausente en 21 de 85 recolecciones
  Última vez vista: 2025-04-15 09:01:54
Estación ID 36 - Plaza Ramales
  Dirección: Plaza de Ramales n.1
  Ausente en 10 de 85 recolecciones
  Última vez vista: 2025-04-14 13:01:41
Estación ID 127 - Mercado Municipal de Vallehermoso
  Dirección: Calle Fernando el Catolico n. 19
  Ausente en 1 de 85 recolecciones
  Última vez vista: 2025-04-15 09:01:54

In [13]:
#Para continuar con la idea, verificaremos las estaciones cuyo status = 0 (inactivo) están presentes en el ultimo timestamp del dataframe
# Identificar el último timestamp en el dataset
ultimo_timestamp = df['timestamp'].max()
print(f"Último timestamp en el dataset: {ultimo_timestamp}")

# Filtrar los datos del último timestamp
df_ultimo = df[df['timestamp'] == ultimo_timestamp]

# Filtrar las estaciones inactivas (status = 0)
estaciones_inactivas = df_ultimo[df_ultimo['status'] == 0]

# Mostrar el número de estaciones inactivas
print(f"Número de estaciones inactivas: {len(estaciones_inactivas)}")

# Mostrar información relevante de las estaciones inactivas
if len(estaciones_inactivas) > 0:
    # Seleccionar columnas relevantes
    columnas = ['station_id', 'name', 'address', 'total_bases', 'available_bikes', 'free_bases', 'status']

    # Mostrar información
    print("\nEstaciones inactivas en el último timestamp:")
    print(estaciones_inactivas[columnas])

else:
    print("No hay estaciones inactivas en el último timestamp.")

Último timestamp en el dataset: 2025-04-15 09:01:54
Número de estaciones inactivas: 0
No hay estaciones inactivas en el último timestamp.


In [14]:
# Ordenar los timestamps únicos de forma descendente
timestamps = sorted(df['timestamp'].unique(), reverse=True)

# Obtener los últimos 10 timestamps
ultimos_10_timestamps = timestamps[:10]

# Filtrar el DataFrame para los últimos 10 timestamps y las estaciones con ceros
estaciones_cero_ultimos_10_timestamps = df[
    (df['timestamp'].isin(ultimos_10_timestamps)) &
    (df['active_bases'] == 0) &
    (df['available_bikes'] == 0) &
    (df['free_bases'] == 0)
][['name', 'public_station_id']]

# Agrupar por nombre e ID y contar las ocurrencias
conteo_estaciones_cero = estaciones_cero_ultimos_10_timestamps.groupby(['name', 'public_station_id']).size().reset_index(name='frecuencia')

print("\nFrecuencia de estaciones con active_bases, available_bikes y free_bases igual a 0 en los últimos 10 timestamps:")
for index, row in conteo_estaciones_cero.iterrows():
    print(f"Estación {row['name']} (ID: {row['public_station_id']}): {row['frecuencia']} ocasiones")


Frecuencia de estaciones con active_bases, available_bikes y free_bases igual a 0 en los últimos 10 timestamps:
Estación Conde de Romanones (ID: 40): 10 ocasiones
Estación Jacinto Benavente (ID: 34): 10 ocasiones
Estación Metro Anton Martin (ID: 41): 10 ocasiones
Estación Metro Oporto (ID: 363): 10 ocasiones
Estación Metro Sol (ID: 1): 10 ocasiones
Estación Metro Usera (ID: 384): 10 ocasiones
Estación Palacio de Oriente (ID: 24): 10 ocasiones
Estación Plaza de Celenque A (ID: 25A): 10 ocasiones
Estación Plaza de Celenque B (ID: 25B): 10 ocasiones
Estación Plaza de San Ildefonso (ID: 55): 10 ocasiones
Estación Plaza de San Miguel (ID: 9): 10 ocasiones
Estación Plaza de Santa Ana (ID: 52): 10 ocasiones
Estación Plaza del Cordon (ID: 35): 10 ocasiones


In [15]:
# Filtrar el DataFrame para las estaciones que alguna vez tuvieron los tres valores en cero
estaciones_con_cero = df[
    (df['active_bases'] == 0) &
    (df['available_bikes'] == 0) &
    (df['free_bases'] == 0)
][['name', 'public_station_id']].drop_duplicates()

# Función para encontrar la última vez con datos reales para una estación
def encontrar_ultima_vez_con_datos(row):
    nombre = row['name']
    station_id = row['public_station_id']
    datos_reales = df[
        (df['name'] == nombre) &
        (df['public_station_id'] == station_id) &
        ((df['active_bases'] != 0) | (df['available_bikes'] != 0) | (df['free_bases'] != 0))
    ].sort_values(by='timestamp', ascending=False)
    if not datos_reales.empty:
        return datos_reales.iloc[0]['timestamp']
    else:
        return None  # Si nunca tuvo datos reales

# Aplicar la función a cada estación que tuvo ceros
estaciones_con_cero['ultima_vez_con_datos'] = estaciones_con_cero.apply(encontrar_ultima_vez_con_datos, axis=1)

# Contar la frecuencia total de las veces que tuvieron los tres valores en cero
conteo_ceros = df[
    (df['active_bases'] == 0) &
    (df['available_bikes'] == 0) &
    (df['free_bases'] == 0)
].groupby(['name', 'public_station_id']).size().reset_index(name='frecuencia_ceros')

# Combinar los resultados
resultado = pd.merge(estaciones_con_cero, conteo_ceros, on=['name', 'public_station_id'], how='left')

print("\nFrecuencia total de ceros y última vez con datos reales:")
for index, row in resultado.iterrows():
    print(f"Estación {row['name']} (ID: {row['public_station_id']}):")
    print(f"  - Frecuencia total con datos cero: {row['frecuencia_ceros'] if pd.notna(row['frecuencia_ceros']) else 0} ocasiones")
    print(f"  - Última vez con datos reales: {row['ultima_vez_con_datos'] if row['ultima_vez_con_datos'] else 'Nunca tuvo datos reales después de tener ceros'}")


Frecuencia total de ceros y última vez con datos reales:
Estación Conde de Romanones (ID: 40):
  - Frecuencia total con datos cero: 85 ocasiones
  - Última vez con datos reales: NaT
Estación Plaza de Santa Ana (ID: 52):
  - Frecuencia total con datos cero: 85 ocasiones
  - Última vez con datos reales: NaT
Estación Metro Usera (ID: 384):
  - Frecuencia total con datos cero: 85 ocasiones
  - Última vez con datos reales: NaT
Estación Metro Oporto (ID: 363):
  - Frecuencia total con datos cero: 85 ocasiones
  - Última vez con datos reales: NaT
Estación Palacio de Oriente (ID: 24):
  - Frecuencia total con datos cero: 49 ocasiones
  - Última vez con datos reales: 2025-04-11 07:00:54
Estación Plaza Ramales (ID: 36):
  - Frecuencia total con datos cero: 39 ocasiones
  - Última vez con datos reales: 2025-04-11 07:00:54
Estación Metro Anton Martin (ID: 41):
  - Frecuencia total con datos cero: 49 ocasiones
  - Última vez con datos reales: 2025-04-11 07:00:54
Estación Plaza de San Ildefonso (ID

## CONSIDERACIONES PARA LA LIMPIEZA Y TRATAMIENTO DE LOS DATOS

Durante el análisis del periodo de recolección de datos, se identificó una inconsistencia en el número de estaciones reportadas. Inicialmente, se esperaba obtener datos de 628 estaciones, sin embargo, este número disminuyó a 625 durante el periodo de recolección. Al comparar la lista inicial de estaciones con las efectivamente capturadas, se identificaron tres estaciones ausentes. Esta discrepancia motivó una investigación para comprender la causa de esta pérdida de datos.

Adicionalmente, se observó que la columna `status` no reflejaba la verdadera disponibilidad de las estaciones, ya que algunas mostraban un estado activo a pesar de tener valores 0 en las columnas `active_bases`, `available_bikes` y `free_bases`. Para investigar esta inconsistencia, se realizó una consulta a la página web oficial de BiciMAD, específicamente en el apartado del **Mapa de estaciones**. En este mapa, las estaciones que mostraban consistentemente valores 0 en los datos recolectados aparecían marcadas como **"Desactivado"**, lo que sugería que su disponibilidad real no se correspondía con el valor de la columna `status`.

Para comprender aún más la causa de la ausencia de datos y el estado "Desactivado" de estas estaciones, se realizó una investigación de campo. En una de las estaciones que consistentemente aparecía ausente en los datos recolectados, se encontró un cartel informativo adjunto a la estación que enunciaba:

> **AVISO IMPORTANTE- BiciMAD**
>
> **Cierre temporal de estaciones por Semana Santa**
>
> Con motivo de la celebración de la semana santa, algunas estaciones de bicimad
> permaneceran cerradas desde el viernes 11 de abril hasta el domingo 20 de abril de 2025
> (Ambos inclusive).
>
> 35/35/9/33/31/24/25a/25b/32/34/40/41/28/39/1/55/52

La investigación reveló que las tres estaciones ausentes en la recolección y aquellas que mostraban consistentemente valores cero en las columnas de disponibilidad coincidían con las estaciones listadas como temporalmente cerradas por Semana Santa tanto en el aviso encontrado en campo como en la información del mapa de estaciones de la página web de BiciMAD.

**Es importante destacar que, durante el periodo de seguimiento de los datos, se observó un caso particular donde una estación específica desapareció por completo de los datos recolectados y también dejó de aparecer en el mapa de estaciones de la página web de BiciMAD. Sin embargo, unos días después, esta misma estación volvió a aparecer en los datos y en el mapa. Esta observación sugiere que, además de los cierres programados por eventos como la Semana Santa, pueden existir desactivaciones temporales de estaciones por motivos como mantenimiento o reparación, lo que también podría explicar algunas de las ausencias o los periodos con datos en cero.**

La decisión que tomé para el tratamiento de éstos datos fue la siguiente:

* Agregar las estaciones ausentes con valor 0 en las columnas `active_bases`, `available_bikes` y `free_bases`. Se consideró que mantener explícitamente estas estaciones con valor 0 permite diferenciarlas de estaciones que podrían tener datos reales en otros momentos y refleja su estado de inoperatividad durante el periodo de cierre o por otras causas temporales.
* Crear una nueva columna llamada `real_status` cuyos únicos valores sean: `active`, `inactive` y `closed_for_schedule`. La columna `real_status` se creó asignando el valor `'inactive'` a las estaciones que consistentemente mostraban valores 0 en las columnas de disponibilidad y que coincidían con las estaciones listadas en el aviso de cierre temporal y el mapa web, o que desaparecieron temporalmente. Las estaciones presentes en los datos con valores distintos de cero se marcaron como `'active'`. Se creó el valor `'closed_for_schedule'` para reflejar explícitamente el motivo del estado inactivo de estas estaciones durante el periodo de Semana Santa, diferenciándolo de posibles inactividades por mantenimiento u otras razones.
* Fijar los formatos de datos correspondientes para cada columna.

El periodo de recolección de datos abarcó las fechas del cierre temporal anunciado, lo que justifica el estado inactivo de estas estaciones. La observación de la estación que desapareció y reapareció subraya la importancia de no depender únicamente de la columna `status` para determinar la operatividad real de las estaciones. Al considerar la disponibilidad real en análisis posteriores, se tendrá en cuenta el estado `closed_for_schedule` y las posibles inactividades por mantenimiento u otras razones temporales para evitar interpretaciones erróneas sobre la operatividad general del sistema BiciMAD. Una limitación de este enfoque es la dificultad para determinar la causa exacta de todas las ausencias o periodos con datos en cero más allá del cierre programado por Semana Santa.

In [16]:
# 1. Creamos las filas faltantes con los datos en "0" a través de la función merge.

# Primero identificamos todos los timestamps y estaciones únicas
all_timestamps = df['timestamp'].unique()
all_stations = df['public_station_id'].unique()

# Creamos una estructura para todas las combinaciones posibles de timestamp y estación

# Uso más eficiente del producto cartesiano con MultiIndex
index = pd.MultiIndex.from_product([all_timestamps, all_stations],
                                  names=['timestamp', 'public_station_id'])
complete_df = pd.DataFrame(index=index).reset_index()

# Hacemos un merge con el DataFrame original
merged_df = pd.merge(
    complete_df,
    df,
    on=['timestamp', 'public_station_id'],
    how='left'
)

# Para las filas que no existían (NaN), rellenamos con ceros los valores numéricos
numeric_columns = ['active_bases', 'available_bikes', 'free_bases', 'total_bases', 'reservations']
for col in numeric_columns:
    if col in merged_df.columns:  # Asegurarse de que la columna existe
        merged_df[col] = merged_df[col].fillna(0).astype(int)  # Convertimos a enteros para eficiencia

# Para las columnas no numéricas, podemos llenarlas con valores adecuados
merged_df['status'] = merged_df['status'].fillna(0).astype(int)  # Status 0 para las ausentes

# Rellenar la columna 'name' y otras columnas usando información disponible para cada estación
station_info = df.drop_duplicates('public_station_id').set_index('public_station_id')
station_info_columns = ['name', 'address', 'latitude', 'longitude', 'station_id']

for column in station_info_columns:
    if column in df.columns:
        # Usamos un diccionario para el mapeo (más eficiente)
        column_dict = station_info[column].to_dict()
        merged_df[column] = merged_df['public_station_id'].map(column_dict)

# Verificamos que hemos completado correctamente
print(f"DataFrame original: {len(df)} registros")
print(f"DataFrame completo: {len(merged_df)} registros")
print(f"Teóricamente deberíamos tener {len(all_timestamps) * len(all_stations)} registros")

# Aseguramos que no haya NaN en columnas críticas
missing_values = merged_df.isnull().sum()
print("\nValores faltantes después de la completación:")
print(missing_values[missing_values > 0])

# Calculamos cuántos registros fueron añadidos sin crear una columna adicional
added_records = len(merged_df) - len(df)
print(f"\nRegistros añadidos: {added_records}")
print(f"Registros originales: {len(df)}")

DataFrame original: 53210 registros
DataFrame completo: 53380 registros
Teóricamente deberíamos tener 53380 registros

Valores faltantes después de la completación:
Series([], dtype: int64)

Registros añadidos: 170
Registros originales: 53210


In [17]:
#Transformar el df que hemos creado a través de merge como el df principal
df_copy = df.copy()
df = merged_df
print(df.describe())

                           timestamp    station_id     longitude  \
count                          53380  53380.000000  53380.000000   
mean   2025-04-11 21:01:00.576470528   1907.514331     -3.682495   
min              2025-04-08 09:00:01   1406.000000     -3.784627   
25%              2025-04-10 03:00:37   1569.750000     -3.708494   
50%              2025-04-11 21:01:02   2011.500000     -3.689462   
75%              2025-04-13 15:01:30   2172.250000     -3.660259   
max              2025-04-15 09:01:54   2397.000000     -3.548534   
std                              NaN    323.772361      0.042466   

           latitude   total_bases  active_bases  available_bikes  \
count  53380.000000  53380.000000  53380.000000     53380.000000   
mean      40.424690     23.743987     23.551574        10.839266   
min       40.332546      0.000000      0.000000         0.000000   
25%       40.398812     23.000000     23.000000         4.000000   
50%       40.423896     23.000000     23.000000

In [18]:
# Creamos la nueva columna con el estado real de las estaciones

# Lista de IDs de estaciones cerradas por programación (Semana Santa 2025)
scheduled_closure_ids = ['35', '9', '33', '31', '24', '25a', '25b', '32', '34',
                         '40', '41', '28', '39', '1', '55', '52']

# Definir el período de cierre programado
closure_start = pd.to_datetime('2025-04-11')
closure_end = pd.to_datetime('2025-04-20')

# Crear condiciones para cada estado de forma vectorizada
is_scheduled_closure = (
    df['public_station_id'].isin(scheduled_closure_ids) &
    (df['timestamp'] >= closure_start) &
    (df['timestamp'] <= closure_end)
)

# Nueva condición: estaciones con los tres valores en cero
is_zero_values = (
    (df['active_bases'] == 0) &
    (df['available_bikes'] == 0) &
    (df['free_bases'] == 0)
)

# Aplicar las condiciones para crear la columna real_status
df['real_status'] = 'active'  # Valor predeterminado
df.loc[is_zero_values, 'real_status'] = 'inactive'  # Marcar estaciones con valores en cero como inactive
df.loc[is_scheduled_closure, 'real_status'] = 'closed_for_schedule'  # Prioridad para cierres programados

# Verificación rápida de la distribución de estados
status_counts = df['real_status'].value_counts()
print("Distribución de estados en el dataset:")
print(status_counts)

# Verificar específicamente las estaciones con todos los valores en cero
zero_values_stations = df[is_zero_values].groupby(['public_station_id', 'name']).size().reset_index(name='count')
print(f"\nHay {len(zero_values_stations)} estaciones con todos los valores en cero")

# Verificar estaciones con cierre programado que también tienen valores en cero
scheduled_and_zero = df[
    is_scheduled_closure & is_zero_values
].groupby(['public_station_id', 'name']).size().reset_index(name='count')

print(f"\nHay {len(scheduled_and_zero)} estaciones con cierre programado que también tienen todos los valores en cero")
if len(scheduled_and_zero) > 0:
    print("Estas estaciones son:")
    display(scheduled_and_zero)

Distribución de estados en el dataset:
real_status
active                 52288
closed_for_schedule      689
inactive                 403
Name: count, dtype: int64

Hay 19 estaciones con todos los valores en cero

Hay 12 estaciones con cierre programado que también tienen todos los valores en cero
Estas estaciones son:


,public_station_id,name,count
0,1,Metro Sol,49
1,24,Palacio de Oriente,49
2,31,Mayor,49
3,32,Plaza de la Provincia,49
4,33,Puerta del Sol,48
5,34,Jacinto Benavente,49
6,35,Plaza del Cordon,48
7,40,Conde de Romanones,53
8,41,Metro Anton Martin,49
9,52,Plaza de Santa Ana,53


In [19]:
#Hacemos las verificaciones del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53380 entries, 0 to 53379
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   timestamp          53380 non-null  datetime64[ns]
 1   public_station_id  53380 non-null  object        
 2   station_id         53380 non-null  int64         
 3   name               53380 non-null  object        
 4   address            53380 non-null  object        
 5   longitude          53380 non-null  float64       
 6   latitude           53380 non-null  float64       
 7   total_bases        53380 non-null  int64         
 8   active_bases       53380 non-null  int64         
 9   available_bikes    53380 non-null  int64         
 10  free_bases         53380 non-null  int64         
 11  reservations       53380 non-null  int64         
 12  status             53380 non-null  int64         
 13  real_status        53380 non-null  object        
dtypes: dat

In [20]:
df.describe()

,timestamp,station_id,longitude,latitude,total_bases,active_bases,available_bikes,free_bases,reservations,status
count,53380,53380.000000,53380.000000,53380.000000,53380.000000,53380.000000,53380.000000,53380.000000,53380.0,53380.000000
mean,2025-04-11 21:01:00.576470528,1907.514331,-3.682495,40.424690,23.743987,23.551574,10.839266,12.712308,0.0,0.996815
min,2025-04-08 09:00:01,1406.000000,-3.784627,40.332546,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,2025-04-10 03:00:37,1569.750000,-3.708494,40.398812,23.000000,23.000000,4.000000,7.000000,0.0,1.000000
50%,2025-04-11 21:01:02,2011.500000,-3.689462,40.423896,23.000000,23.000000,10.000000,13.000000,0.0,1.000000
75%,2025-04-13 15:01:30,2172.250000,-3.660259,40.449096,24.000000,26.000000,16.000000,19.000000,0.0,1.000000
max,2025-04-15 09:01:54,2397.000000,-3.548534,40.515720,43.000000,54.000000,50.000000,38.000000,0.0,1.000000
std,NaN,323.772361,0.042466,0.035980,2.694973,4.393723,7.897825,7.099028,0.0,0.056344


In [21]:
df.shape

(53380, 14)

In [22]:
#Verificamos los valores unicos
df['timestamp'].nunique()

85

In [23]:
df['public_station_id'].nunique()

628

In [24]:
df['real_status'].nunique()

3

In [25]:
# Verificar si 'total_bases' y 'active_bases' son diferentes en df
bases_diferentes_df = df[df['total_bases'] != df['active_bases']]

print(f"Número de filas en df donde 'total_bases' es diferente de 'active_bases': {len(bases_diferentes_df)}")

if len(bases_diferentes_df) > 0:
    print("\nEjemplos de filas en df donde 'total_bases' y 'active_bases' son diferentes:")
    print(bases_diferentes_df[['timestamp', 'public_station_id', 'name', 'total_bases', 'active_bases']].head())

# También verificar si 'active_bases' es mayor que 'total_bases' en df
active_mayor_total_df = df[df['active_bases'] > df['total_bases']]

print(f"\nNúmero de filas en df donde 'active_bases' es mayor que 'total_bases': {len(active_mayor_total_df)}")

if len(active_mayor_total_df) > 0:
    print("\nEjemplos de filas en df donde 'active_bases' es mayor que 'total_bases':")
    print(active_mayor_total_df[['timestamp', 'public_station_id', 'name', 'total_bases', 'active_bases']].head())

Número de filas en df donde 'total_bases' es diferente de 'active_bases': 24385

Ejemplos de filas en df donde 'total_bases' y 'active_bases' son diferentes:
             timestamp public_station_id                            name  \
11 2025-04-08 09:00:01               542    Intercambiador de Valdebebas   
14 2025-04-08 09:00:01               289  Luis de Salazar - Padre Claret   
15 2025-04-08 09:00:01               402                     Berrocal, 9   
16 2025-04-08 09:00:01               543                   Tomas Redondo   
29 2025-04-08 09:00:01               283        Museo Nacional del Prado   

    total_bases  active_bases  
11           23            17  
14           23            22  
15           27            31  
16           23            22  
29           23            24  

Número de filas en df donde 'active_bases' es mayor que 'total_bases': 8905

Ejemplos de filas en df donde 'active_bases' es mayor que 'total_bases':
             timestamp public_station_id  

In [26]:
#Verificamos si hay consistencia entre el numero de bases en uso con las disponibles
inconsistent_bases = df[df['available_bikes'] + df['free_bases'] != df['active_bases']]
print(f"Número de filas con bases inconsistentes: {len(inconsistent_bases)}")
if len(inconsistent_bases) > 0:
    print(inconsistent_bases.head())

Número de filas con bases inconsistentes: 0


#CONSIDERACIONES PARA ABORDAR LAS INCONSITENCIAS ENTRE LAS BASES TOTALES Y LAS BASES ACTIVAS

He revisado las verificaciones que has realizado y veo una inconsistencia importante en los datos: en muchos casos (24,385 registros), el valor de total_bases difiere de active_bases, y en algunos casos (8,905 registros), active_bases es incluso mayor que total_bases, lo cual no debería ocurrir lógicamente.

Sin embargo, también has comprobado que available_bikes + free_bases = active_bases para todos los registros, lo que indica consistencia interna entre estas tres variables.

Lo que haremos es tomar en cuenta a Active_bases para analisis de ocupación y el total_bases para un analisis de proporcionalidad con las bases activas.

In [27]:
#Una vez terminada la verificación eliminamos las columnas que no usaremos para hacer el analisis de datos
df.drop(columns=['reservations'], inplace=True)
df.head()

,timestamp,public_station_id,station_id,name,address,longitude,latitude,total_bases,active_bases,available_bikes,free_bases,status,real_status
0,2025-04-08 09:00:01,5,1409,Fuencarral,Calle Fuencarral n. 106,-3.702135,40.428521,27,27,17,10,1,active
1,2025-04-08 09:00:01,8,1412,Metro Alonso Martinez,Plaza de Alonso Martinez n. 6,-3.695440,40.427868,24,24,7,17,1,active
2,2025-04-08 09:00:01,10,1414,Marques de la Ensenada,"Calle Marques de la Ensenada (junto a edif, Ce...",-3.692104,40.425403,23,23,3,20,1,active
3,2025-04-08 09:00:01,11,1415,Plaza del Dos de Mayo,Calle San Andres n. 18 - 20,-3.703600,40.427000,23,23,1,22,1,active
4,2025-04-08 09:00:01,609,2286,Metro Ronda de la Comunicacion,"Ronda de la Comunicacion, s/n",-3.662689,40.515720,23,23,1,22,1,active


In [28]:
# Agrupamos por public_station_id y obtenemos los nombres y direcciones únicos
verificacion = df.groupby('public_station_id').agg(
    nombres=('name', 'unique'),
    direcciones=('address', 'unique')
)

# Identificamos los public_station_id con más de un nombre asociado
inconsistencias_nombre = {
    id: {'nombres': data['nombres'], 'direcciones': data['direcciones']}
    for id, data in verificacion.iterrows() if len(data['nombres']) > 1
}

if inconsistencias_nombre:
    print("Se encontraron inconsistencias de nombre para las siguientes estaciones:")
    df_inconsistencias_nombre = pd.DataFrame([(id, data['nombres'], data['direcciones']) for id, data in inconsistencias_nombre.items()],
                                            columns=['public_station_id', 'nombres_asociados', 'direcciones_asociadas'])
    display(df_inconsistencias_nombre)
else:
    print("Cada public_station_id corresponde a un único nombre de estación.")

# Verificamos si hay nombres duplicados para diferentes public_station_id y mostramos las direcciones
verificacion_inversa = df.groupby('name').agg(
    ids=('public_station_id', 'unique'),
    direcciones=('address', 'unique')
)

inconsistencias_inversas = {
    nombre: {'ids': data['ids'], 'direcciones': data['direcciones']}
    for nombre, data in verificacion_inversa.iterrows() if len(data['ids']) > 1
}

if inconsistencias_inversas:
    print("\nSe encontraron nombres asociados a múltiples public_station_id:")
    df_inconsistencias_inversas = pd.DataFrame([(nombre, data['ids'], data['direcciones']) for nombre, data in inconsistencias_inversas.items()],
                                                 columns=['nombre_estacion', 'public_station_ids_asociados', 'direcciones_asociadas'])
    display(df_inconsistencias_inversas)
else:
    print("\nCada nombre de estación está asociado a un único public_station_id.")

# Corrección específica para "Metro Diego de Leon"
nombre_a_corregir = "Metro Diego de Leon"
if nombre_a_corregir in inconsistencias_inversas:
    ids_a_corregir = inconsistencias_inversas[nombre_a_corregir]['ids']
    direcciones_a_corregir = inconsistencias_inversas[nombre_a_corregir]['direcciones']

    print(f"\nIDs asociados a '{nombre_a_corregir}': {ids_a_corregir}")
    print(f"Direcciones asociadas a '{nombre_a_corregir}': {direcciones_a_corregir}")

    for i, station_id in enumerate(ids_a_corregir):
        if i == 0:
            nuevo_nombre = f"{nombre_a_corregir} A"
        elif i == 1:
            nuevo_nombre = f"{nombre_a_corregir} B"
        else:
            nuevo_nombre = f"{nombre_a_corregir} {chr(ord('C') + i - 2)}"

        df.loc[df['public_station_id'] == station_id, 'name'] = nuevo_nombre

    print(f"\nNombres de '{nombre_a_corregir}' actualizados en el DataFrame.")

    # Volvemos a verificar la correspondencia DESPUÉS de la corrección
    print("\nVerificando correspondencia DESPUÉS de la corrección para 'Metro Diego de Leon'...")
    verificacion_actualizada = df.groupby('public_station_id').agg(
        nombres=('name', 'unique'),
        direcciones=('address', 'unique')
    )
    inconsistencias_nombre_actualizado = {
        id: {'nombres': data['nombres'], 'direcciones': data['direcciones']}
        for id, data in verificacion_actualizada.iterrows() if len(data['nombres']) > 1
    }
    if nombre_a_corregir in [item for sublist in verificacion_actualizada['nombres'] for item in sublist]:
        print(f"Advertencia: Todavía hay referencias a '{nombre_a_corregir}' en los nombres únicos por estación (podría ser parte de otro nombre).")

    verificacion_inversa_actualizada = df.groupby('name').agg(
        ids=('public_station_id', 'unique'),
        direcciones=('address', 'unique')
    )
    inconsistencias_inversas_actualizado = {
        nombre: {'ids': data['ids'], 'direcciones': data['direcciones']}
        for nombre, data in verificacion_inversa_actualizada.iterrows() if len(data['ids']) > 1
    }
    if nombre_a_corregir in inconsistencias_inversas_actualizado:
        print(f"Error: Todavía hay múltiples IDs asociados a '{nombre_a_corregir}' después de la corrección.")
    elif f"{nombre_a_corregir} A" in inconsistencias_inversas_actualizado or f"{nombre_a_corregir} B" in inconsistencias_inversas_actualizado:
        print(f"Corrección para '{nombre_a_corregir}' parece exitosa.")
    else:
        print(f"No se encontraron inconsistencias para '{nombre_a_corregir}' después de la corrección.")

else:
    print(f"\nNo se encontraron inconsistencias para '{nombre_a_corregir}'.")

print("\nPrimeras filas del DataFrame con los nombres actualizados:")
print(df.head())

Cada public_station_id corresponde a un único nombre de estación.

Se encontraron nombres asociados a múltiples public_station_id:


,nombre_estacion,public_station_ids_asociados,direcciones_asociadas
0,Metro Diego de Leon,"[144, 191]","[Calle de Juan Bravo, 51 , Calle de Alfonso He..."



IDs asociados a 'Metro Diego de Leon': ['144' '191']
Direcciones asociadas a 'Metro Diego de Leon': ['Calle de Juan Bravo, 51 ' 'Calle de Alfonso Heredia, 2 ']

Nombres de 'Metro Diego de Leon' actualizados en el DataFrame.

Verificando correspondencia DESPUÉS de la corrección para 'Metro Diego de Leon'...
No se encontraron inconsistencias para 'Metro Diego de Leon' después de la corrección.

Primeras filas del DataFrame con los nombres actualizados:
            timestamp public_station_id  station_id  \
0 2025-04-08 09:00:01                 5        1409   
1 2025-04-08 09:00:01                 8        1412   
2 2025-04-08 09:00:01                10        1414   
3 2025-04-08 09:00:01                11        1415   
4 2025-04-08 09:00:01               609        2286   

                             name  \
0                      Fuencarral   
1           Metro Alonso Martinez   
2          Marques de la Ensenada   
3           Plaza del Dos de Mayo   
4  Metro Ronda de la Comunic

In [29]:
# Detección de Inconsistencias Directas en Columnas Numéricas

# Lista de columnas numéricas a verificar
numeric_columns = ['active_bases', 'available_bikes', 'free_bases', 'total_bases']

# 1. Estadísticas Descriptivas
print("### 1. Estadísticas Descriptivas de las Columnas Numéricas:\n")
print(df[numeric_columns].describe().to_markdown())
print("\n")

# 2. Verificación de Valores Negativos (Lógicamente Imposibles)
print("### 2. Verificación de Valores Negativos:\n")
for column in numeric_columns:
    negativos = df[df[column] < 0]
    if not negativos.empty:
        print(f"**Se encontraron {len(negativos)} valores negativos en la columna '{column}':**")
        print(negativos[['timestamp', 'public_station_id', 'name', column]].head().to_markdown(index=False))
        print("\n")
    else:
        print(f"No se encontraron valores negativos en la columna '{column}'.\n")

# 3. Verificación de 'available_bikes' > 'total_bases' (Lógicamente Imposible)
print("### 3. Verificación de 'available_bikes' > 'total_bases':\n")
bicicletas_mayor_total = df[df['available_bikes'] > df['total_bases']]
if not bicicletas_mayor_total.empty:
    print(f"**Se encontraron {len(bicicletas_mayor_total)} filas donde 'available_bikes' es mayor que 'total_bases':**")
    print(bicicletas_mayor_total[['timestamp', 'public_station_id', 'name', 'available_bikes', 'total_bases']].head().to_markdown(index=False))
    print("\n")
else:
    print("No se encontraron filas donde 'available_bikes' es mayor que 'total_bases'.\n")

# 4. Verificación de 'free_bases' > 'total_bases' (Lógicamente Imposible)
print("### 4. Verificación de 'free_bases' > 'total_bases':\n")
bases_libres_mayor_total = df[df['free_bases'] > df['total_bases']]
if not bases_libres_mayor_total.empty:
    print(f"**Se encontraron {len(bases_libres_mayor_total)} filas donde 'free_bases' es mayor que 'total_bases':**")
    print(bases_libres_mayor_total[['timestamp', 'public_station_id', 'name', 'free_bases', 'total_bases']].head().to_markdown(index=False))
    print("\n")
else:
    print("No se encontraron filas donde 'free_bases' es mayor que 'total_bases'.\n")

# 5. Conteo de valores cero en columnas numéricas
print("### 5. Conteo de Valores Cero en Columnas Numéricas:\n")
for column in numeric_columns:
    ceros = df[df[column] == 0]
    print(f"Número de ceros en la columna '{column}': {len(ceros)}")
print("\n")

### 1. Estadísticas Descriptivas de las Columnas Numéricas:

|       |   active_bases |   available_bikes |   free_bases |   total_bases |
|:------|---------------:|------------------:|-------------:|--------------:|
| count |    53380       |       53380       |  53380       |   53380       |
| mean  |       23.5516  |          10.8393  |     12.7123  |      23.744   |
| std   |        4.39372 |           7.89782 |      7.09903 |       2.69497 |
| min   |        0       |           0       |      0       |       0       |
| 25%   |       23       |           4       |      7       |      23       |
| 50%   |       23       |          10       |     13       |      23       |
| 75%   |       26       |          16       |     19       |      24       |
| max   |       54       |          50       |     38       |      43       |


### 2. Verificación de Valores Negativos:

No se encontraron valores negativos en la columna 'active_bases'.

No se encontraron valores negativos en la colum

In [30]:
# Corregir los casos donde 'available_bikes' es mayor que 'total_bases'
df.loc[df['available_bikes'] > df['total_bases'], 'available_bikes'] = df['total_bases']

print(df[['available_bikes', 'free_bases']].head())

   available_bikes  free_bases
0               17          10
1                7          17
2                3          20
3                1          22
4                1          22


In [31]:
# Análisis de Valores Cero en Columnas Numéricas según Estado Operativo

# Lista de columnas numéricas a verificar.
numeric_columns = ['active_bases', 'available_bikes', 'free_bases', 'total_bases']

print("### Análisis de Valores Cero con Consideración de 'real_status':\n")

for column in numeric_columns:
    ceros_df = df[df[column] == 0]
    total_ceros = len(ceros_df)
    print(f"**Columna: '{column}' - Total de valores cero: {total_ceros}**")

    if 'real_status' in df.columns:
        ceros_activos = ceros_df[ceros_df['real_status'] == 'active']
        num_ceros_activos = len(ceros_activos)
        porcentaje_ceros_activos = (num_ceros_activos / total_ceros) * 100 if total_ceros > 0 else 0
        print(f"  - Valores cero en estaciones 'active': {num_ceros_activos} ({porcentaje_ceros_activos:.2f}%)")

        ceros_inactivos = ceros_df[ceros_df['real_status'].isin(['inactive', 'closed_for_schedule'])]
        num_ceros_inactivos = len(ceros_inactivos)
        porcentaje_ceros_inactivos = (num_ceros_inactivos / total_ceros) * 100 if total_ceros > 0 else 0
        print(f"  - Valores cero en estaciones 'inactive'/'closed_for_schedule': {num_ceros_inactivos} ({porcentaje_ceros_inactivos:.2f}%)")
    else:
        print("  - Columna 'real_status' no encontrada para análisis detallado.")
    print("\n")

# Análisis específico de 'total_bases' en estaciones activas con valor cero
if 'real_status' in df.columns:
    ceros_total_bases_activos = df[(df['total_bases'] == 0) & (df['real_status'] == 'active')]
    num_ceros_total_bases_activos = len(ceros_total_bases_activos)
    print(f"### Análisis Específico de 'total_bases' = 0 en estaciones 'active':\n")
    print(f"Número de filas con 'total_bases' = 0 y 'real_status' = 'active': {num_ceros_total_bases_activos}")
    if num_ceros_total_bases_activos > 0:
        print("Primeras filas de estas estaciones:")
        print(ceros_total_bases_activos[['timestamp', 'public_station_id', 'name', 'total_bases', 'real_status']].head().to_markdown(index=False))
    print("\n")

### Análisis de Valores Cero con Consideración de 'real_status':

**Columna: 'active_bases' - Total de valores cero: 997**
  - Valores cero en estaciones 'active': 0 (0.00%)
  - Valores cero en estaciones 'inactive'/'closed_for_schedule': 997 (100.00%)


**Columna: 'available_bikes' - Total de valores cero: 3658**
  - Valores cero en estaciones 'active': 2654 (72.55%)
  - Valores cero en estaciones 'inactive'/'closed_for_schedule': 1004 (27.45%)


**Columna: 'free_bases' - Total de valores cero: 2376**
  - Valores cero en estaciones 'active': 1376 (57.91%)
  - Valores cero en estaciones 'inactive'/'closed_for_schedule': 1000 (42.09%)


**Columna: 'total_bases' - Total de valores cero: 170**
  - Valores cero en estaciones 'active': 0 (0.00%)
  - Valores cero en estaciones 'inactive'/'closed_for_schedule': 170 (100.00%)


### Análisis Específico de 'total_bases' = 0 en estaciones 'active':

Número de filas con 'total_bases' = 0 y 'real_status' = 'active': 0




In [32]:
# Dataset de estaciones (información estática)
stations_df = df.groupby(['public_station_id', 'station_id', 'name', 'address', 'latitude', 'longitude']).agg({
    'total_bases': 'first',
    'active_bases': 'max'
}).reset_index()

# Dataset de estado de estaciones (información dinámica)
station_status_df = df[['public_station_id', 'timestamp', 'active_bases', 'available_bikes',
                        'free_bases', 'status', 'real_status']]

# Guardar ambos datasets
stations_df.to_csv('bicimad_stations.csv', index=False)
station_status_df.to_csv('bicimad_station_status.csv', index=False)

# Guardar el dataset completo
df.to_csv('bicimad_full.csv', index=False)

print("Datasets guardados exitosamente como:")
print("- bicimad_stations.csv")
print("- bicimad_station_status.csv")
print("- bicimad_full.csv")

Datasets guardados exitosamente como:
- bicimad_stations.csv
- bicimad_station_status.csv
- bicimad_full.csv
